# Train Hierarchical ESN and ESN+SpaRCe

In [1]:
import torch

from echovpr.datasets    import getHiddenRepr, DataSets, Tolerance
from echovpr.networks    import hierESN, getSparsity
from echovpr.experiments import ESN_Exp, getValidationIndices

## Get NetVLAD Hidden Representation and Validation Indices

In [2]:
ds  = 'GardensPoint'
tol = Tolerance[ds]

# Get NetVLAD Hidden Representation
hiddenReprTrain, hiddenReprTest = getHiddenRepr('GardensPoint')
# Get Input and Output Size; First element of shape: (number of images == number of classes == nOutput); Second element:  size of hidden representations
nOutput, nInput = hiddenReprTrain.shape 
# Get Validation Indices
TestInd, ValidationInd = getValidationIndices(1, nOutput)

# hierESN Training

In [3]:
nRes1  = 1000
nRes2  = 1000
nCon   = 10
nTrial = 10
nEpoch = 50
nBatch = 5
lR     = 0.001
alpha1 = 0.6
alpha2 = 0.9
gamma1 = 0.01
gamma2 = 0.01

In [4]:
_hierESN_model_ = lambda randSeed: hierESN(nInput, nOutput, nReservoir1=nRes1, nReservoir2=nRes2, randomSeed = randSeed, device='cpu', useReadout = False,
                                           sparsity1 = getSparsity(nCon, nRes1), sparsity2 = getSparsity(nCon, nRes2), alpha1 = alpha1, alpha2 = alpha2, 
                                           gamma1 = gamma1, gamma2 = gamma2, rho1 = 0.99, rho2 = 0.99)

In [11]:
model   = _hierESN_model_(0)
exp     = ESN_Exp(model, hiddenReprTrain, hiddenReprTest, TestInd, ValidationInd , tol)
results = exp.train_esn(nEpoch=nEpoch, lR=lR, nBatch=nBatch, returnData=True, returnDataAll=False)
results['AccTest'][-1].mean()

0.8399999999999999

# ESN+SpaRCe Training

In [6]:
nRes     = 1000
nCon     = 10
nTrial   = 10
nEpoch   = 50
nBatch   = 5
lR       = 0.0005
alpha1   = 0.6
alpha2   = 0.9
gamma1   = 0.01
gamma2   = 0.01
quantile = 0.4 
lrDiv    = 1000

In [7]:
_hierESN_SPARCE_model_ = lambda randSeed: hierESN(nInput, nOutput, nReservoir1=nRes1, nReservoir2=nRes2, randomSeed = randSeed, device='cpu', useReadout = False,
                                                  sparsity1 = getSparsity(nCon, nRes1), sparsity2 = getSparsity(nCon, nRes2), alpha1 = alpha1, alpha2 = alpha2, 
                                                 gamma1 = gamma1, gamma2 = gamma2, rho1 = 0.99, rho2 = 0.99)

In [12]:
model   = _hierESN_SPARCE_model_(0)
exp     = ESN_Exp(model, hiddenReprTrain, hiddenReprTest, TestInd, ValidationInd , tol)
results = exp.train_sparce(nEpoch=nEpoch, lR=lR, nBatch=nBatch, quantile=quantile, lr_divide_factor=lrDiv, returnData=True, returnDataAll=False)
results['AccTest'][-1].mean()

0.775